In [2]:
import cv2
import numpy as np
import matplotlib.pylab as plt

#--① 크로마키 배경 영상과 합성할 배경 영상 읽기
img1 = cv2.imread('../img/m.jpg')  # 크로마키 배경이 있는 이미지 읽기
img2 = cv2.imread('../img/street.jpg')         # 배경으로 사용할 이미지 읽기

#--② ROI 선택을 위한 좌표 계산
height1, width1 = img1.shape[:2]  # 크로마키 이미지의 높이와 너비를 가져옴
height2, width2 = img2.shape[:2]  # 배경 이미지의 높이와 너비를 가져옴
x = (width2 - width1) // 2        # 배경 이미지의 가로 중앙에 크로마키 이미지를 배치하기 위한 x 좌표 계산
y = height2 - height1             # 배경 이미지의 세로 아래쪽에 크로마키 이미지를 배치하기 위한 y 좌표 계산
w = x + width1                    # 크로마키 이미지가 끝나는 가로 좌표
h = y + height1                   # 크로마키 이미지가 끝나는 세로 좌표

#--③ 크로마키 배경 영상에서 크로마키 영역을 10픽셀 정도로 지정
chromakey = img1[:10, :10, :]      # 크로마키 색상을 얻기 위해 이미지의 왼쪽 위 10x10 픽셀 영역을 선택
offset = 20                        # 색상 범위에 여유를 주기 위한 오프셋 설정

#--④ 크로마키 영역과 영상 전체를 HSV로 변경
hsv_chroma = cv2.cvtColor(chromakey, cv2.COLOR_BGR2HSV)  # 크로마키 영역을 HSV 색상 공간으로 변환
hsv_img = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)          # 크로마키 이미지를 전체적으로 HSV 색상 공간으로 변환

#--⑤ 크로마키 영역의 H값에서 offset 만큼 여유를 두어서 범위 지정
# offset 값은 여러차례 시도 후 결정
chroma_h = hsv_chroma[:,:,0]  # 크로마키 영역에서 H 채널 값만 추출
lower = np.array([chroma_h.min()-offset, 100, 100])  # 크로마키 색상 하한값 설정
upper = np.array([chroma_h.max()+offset, 255, 255])  # 크로마키 색상 상한값 설정

#--⑥ 마스크 생성 및 마스킹 후 합성
mask = cv2.inRange(hsv_img, lower, upper)   # 이미지에서 크로마키 색상 영역을 마스크로 생성
mask_inv = cv2.bitwise_not(mask)            # 마스크를 반전시켜 크로마키 색상이 아닌 영역을 선택
roi = img2[y:h, x:w]                        # 배경 이미지에서 크로마키 이미지가 들어갈 영역을 선택
fg = cv2.bitwise_and(img1, img1, mask=mask_inv)  # 크로마키 이미지에서 크로마키 색상 영역을 제거
bg = cv2.bitwise_and(roi, roi, mask=mask)        # 배경 이미지에서 크로마키 색상 영역만 선택
img2[y:h, x:w] = fg + bg                        # 크로마키 이미지와 배경 이미지를 합성

#--⑦ 결과 출력
cv2.imshow('chromakey', img1)  # 크로마키 이미지 출력
cv2.imshow('added', img2)      # 합성된 이미지 출력
cv2.waitKey()                  # 키 입력 대기
cv2.destroyAllWindows()        # 모든 창 닫기


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'
